# DEM and orthoimage stack processing and visualization
### Examples for BlackSky Easton Glacier test case (n=20)

David Shean  
1/2/23

### To do:
* add triangulation error metrics (mean/std, median/nmad)
* add h and v error metrics

In [ ]:
#Run script to generate relevant stack products (e.g., composite, std, nmad, count)
#!make_dem_stack.sh

In [ ]:
out_prefix = 'dem_composite'

In [ ]:
def plot_dem_stack_results(outdir, out_prefix=out_prefix, hillshade=True):
    #Note: could have grid resolution differences here
    f, axa = plt.subplots(2,3, figsize=(10,7.5), dpi=300)
    axa = axa.ravel()

    ortho_fn = f'{out_prefix}_ortho-tile-0.tif'
    if os.path.exists(ortho_fn):
        ortho_ma = read_array(ortho_fn)
        plot_ar(ortho_ma, ax=axa[0], cmap='gray', clim_perc=(0,100))
        axa[0].set_title('Orthomosaic')
    else:
        axa[0].axis('off')
    
    dem_fn = f'{out_prefix}-tile-0.tif'
    dem_ma = read_array(dem_fn)
    with rio.open(dem_fn) as ds:
        dem_gsd = ds.res[0]
    
    if hillshade:
        hs_fn = f'{out_prefix}-tile-0_hs.tif'
        if hs_fn:
            hs_ma = read_array(hs_fn)
        else:
            dem_ds = gdal.Open(dem_fn)
            hs_ds = gdal.DEMProcessing('', dem_ds, 'hillshade', format='MEM', computeEdges=True)
            hs_ma = np.ma.masked_equal(hs_ds.ReadAsArray(), 0)
        plot_ar(hs_ma, ax=axa[1], cmap='gray', clim_perc=(5,95), cbar=False)
    
    #axa[4].set_title(os.path.split(dem_fn)[-1])
    plot_ar(dem_ma, ax=axa[1], cmap='viridis', alpha=0.5) #label='Elevation (m HAE)'
    axa[1].set_title('DEM composite (m HAE)')
    
    scalebar = ScaleBar(dem_gsd)
    axa[1].add_artist(scalebar)
    
    count_fn = f'{out_prefix}-tile-0-count.tif'
    if os.path.exists(count_fn):
        count_ma = read_array(count_fn)
        plot_ar(count_ma, ax=axa[2], cmap='YlOrRd', clim_perc=(0,100))
        axa[2].set_title('DEM count')
    else:
        axa[2].axis('off')
        
    nmad_fn = f'{out_prefix}-tile-0-nmad.tif'
    if os.path.exists(nmad_fn):
        nmad_ma = read_array(nmad_fn)
        #plot_ar(nmad_ma, ax=axa[4], cmap='inferno', clim_perc=(0,95))
        plot_ar(nmad_ma, ax=axa[4], cmap='inferno', clim=(0,30))
        axa[4].set_title('DEM nmad (m)')
    else:
        axa[4].axis('off')
    
    std_fn = f'{out_prefix}-tile-0-stddev.tif'
    if os.path.exists(std_fn):
        std_ma = read_array(std_fn)
        #plot_ar(std_ma, ax=axa[3], cmap='inferno', clim_perc=(0,95))
        plot_ar(std_ma, ax=axa[3], cmap='inferno', clim=(0,60))
        axa[3].set_title('DEM std (m)')
    else:
        axa[3].axis('off')
    
    #This is not generated by default (requires point2dem --errorimage)
    diff_fn = glob.glob(f'{out_prefix}-tile-0*diff.tif')
    if diff_fn:
        diff_fn = diff_fn[0]
        diff_ma = read_array(diff_fn)
        #Note negative sign here, as these are DEM minus REF, and want REF minus DEM
        #plot_ar(-diff_ma, ax=axa[5], clim=get_clim(diff_ma, symm=True), cmap='RdBu') #label='Refdem diff (m)'
        plot_ar(-diff_ma, ax=axa[5], clim=(-50, 50), cmap='RdBu') #label='Refdem diff (m)'
        axa[5].set_title('COP30 minus DEM (m)')
    else:
        axa[5].axis('off')
    
    #f.suptitle(os.path.split(outdir)[-1])
    f.tight_layout()
    out_fn = f'{out_prefix}_stack_report.png'
    plt.savefig(out_fn, bbox_inches='tight')

In [ ]:
plot_dem_stack_results('.', out_prefix)

In [ ]:
#Create a gallery of preview images
#Can do both orthoimage and DEMs
#!/Users/dshean/src/asp_plot/gallery.py $(ls *ortho/*rpc.tif | sort -t '-' -n -k3)

In [ ]:
#Compute some stats
#Diff over masked refdem
#Diff within 3 km radius
#TriErr stack?